In [1]:
import os.path
from scipy import *
import numpy as np
from numpy import *
import scipy as sp
from numpy import linalg as LA
from scipy import linalg as LA2
import sympy as sympy
import sys as sys
import time
import matplotlib.pyplot as plt
import itertools as it
from fractions import Fraction
from decimal import Decimal
from IPython.core.display import HTML


sys.path.append('/Users/sashacurcic/SashasDirectory/ANAG/FV_MG/')
from Modules import BasicTools as BT
from Modules import WaveTools as WT
from Modules import PlotTools as PT
from Modules import FFTTools as FFTT
from Modules import OperatorTools as OT
from Modules import GridTransferTools as GTT
from Modules import TestTools as TT
from Modules import SolverTools as ST
from Modules import WaveformTools as WFT

display(HTML("<style>pre { white-space: pre !important; }</style>"))
np.set_printoptions( linewidth = 10000, threshold = 100000)

In [2]:
nh_min = 8
refRatio = 2

diff = 'DD'
order = 3
RL = 'L'

L = 1.
locs = [0.5]
epsilons = [1, 0.5]# 11.68]
mus = [1, 0.5]#0.99837]

In [3]:
omegaAMR = BT.Grid(nh_min)
finehalf = list(np.arange(int(nh_min / refRatio)) + int(nh_min / 2))
omegaAMR.AddPatch(refRatio, finehalf)
degFreed = omegaAMR.degFreed
nh_max = omegaAMR.nh_max

omegaF = BT.Grid(nh_max)
omegaC = BT.Grid(nh_min)

wavesAMR = WT.MakeWaves(omegaAMR)
nullspace = OT.FindNullspace(omegaAMR, wavesAMR)
nullspaceBlock = OT.Block(nullspace, var = 2)
restrictOp = GTT.CoarsenOp(omegaAMR)
restrictOpBlock = OT.Block(restrictOp, var = 2)

wavesF = WT.MakeWaves(omegaF)
wavesC = WT.MakeWaves(omegaC)

In [4]:
physicsAMR = BT.PhysProps(omegaAMR, epsilons, mus, locs, L)
c = physicsAMR.cVec
cMatAMR = physicsAMR.cMat

physicsC = BT.PhysProps(omegaC, epsilons, mus, locs, L)
cC = physicsC.cVec
cMatC = physicsC.cMat

physicsF = BT.PhysProps(omegaF, epsilons, mus, locs, L)
cF = physicsF.cVec
cMatF = physicsF.cMat

cVecC = physicsC.cVec
c1 = cVecC[0]
c2 = cVecC[-1]

In [5]:
def CDFace(orderIn):
    errorLoc = 'ERROR:\nOperatorTools:\nCDFace:\n'
    errorMess = ''
    if (orderIn % 2 == 0):
        order = orderIn
    else:
        order = int(orderIn + 1)
    
    stenc = np.zeros(order + 1)
    
    if (order == 2):
        face = np.asarray([1, 1])
    else:
        if (order == 4):
            face = (1. / 6.) * np.asarray([-1, 7, 7, -1])
        else:
            errorMess = 'This program is not designed to handle this order of accuracy for center-difference operators.'
    
    if (errorMess != ''):
        sys.exit(errorLoc + errorMess)
    
    return face

In [6]:
def UDFace(orderIn):
    errorLoc = 'ERROR:\nOperatorTools:\nUDFace:\n'
    errorMess = ''
    if (orderIn % 2 == 0):
        order = int(orderIn + 1)
    else:
        order = orderIn
    
    stenc = np.zeros(order + 1)
    
    if (order == 1):
        face = np.asarray([1])
    else:
        if (order == 3):
            face = (1. / 6.) * np.asarray([-1, 5, 2])
        else:
            if (order == 5):
                face = (1. / 60.) * np.asarray([2, -13, 47, 27, -3])
            else:
                if (order == 7):
                    face = (1. / 420.) * np.asarray([-3, 25, -101, 319, 214, -38, 4])
                else:
                    if (order == 9):
                        face = (1. / 2520.) * np.asarray([4, -41, 199, -641, 1879, 1375, -305, 55, -5])
                    else:
                        errorMess = 'This program is not designed to handle this order of accuracy for forward- and backward-difference operators.'
    
    if (errorMess != ''):
        sys.exit(errorLoc + errorMess)
    
    return face

In [7]:
def DDFace(order):
    stenc = UDFace(order)[::-1]
    return stenc

In [8]:
def FaceOp(omega, order, diff, RL, Ng):
    errorLoc = 'ERROR:\nOperatorTools:\nMakeSpaceDeriv:\n'
    errorMess = ''
    if (diff == 'C' or diff == 'CD'):
        stenc = OT.CDStencil(order)
        if (order % 2 == 0):
            orderStenc = order
        else:
            orderStenc = int(order + 1)
        off = int(orderStenc / 2)
        loBound = -off / 2.
        hiBound = off / 2.
    else:
        orderStenc = order
        if (order % 2 == 0):
            orderStenc = order # int(order + 1)
        else:
            orderStenc = int(order - 1) # order
        off = int(orderStenc / 2) # int((orderStenc + 1) / 2)
        if (diff == 'U' or diff == 'UD'):
            stenc = UDFace(order) # OT.UDStencil(order)
            loBound = -off / 2.
            hiBound = off / 2. #(off - 1.) / 2.
        else:
            if (diff == 'D' or diff == 'DD'):
                stenc = DDFace(order) # OT.DDStencil(order)
                off = int(off - 1)
                loBound = -(off + 1.) / 2. # -off / 2.
                hiBound = (off + 1.) / 2.
            else:
                errorMess = 'Invalid entry for variable diff. Must be \'C\', \'U\', \'D\' \'CD\', \'UD\', or \'DD\'.'
    if (errorMess != ''):
        sys.exit(errorLoc + errorMess)
    
#     stenc = np.ones(orderStenc + 1)
    
    degFreed = omega.degFreed
    hs = omega.h
    
    spots = np.roll(hs, -1) - hs
    
    if (all(spots == 0)):
        p = []
        q = []
        NU = False
    else:
        # Index before fine-coarse interface
        p = np.where(spots > 0)[0][0]
        # Index before coarse-fine interface
        q = np.where(spots < 0)[0][0]
        NU = True
    
    polyStencSet = [[] for i in range(orderStenc)]
    cellFaces = np.linspace(loBound, hiBound, num = orderStenc + 1)
    zeroLoc = np.where(cellFaces == 0)[0][0]
    cellFaces = np.delete(cellFaces, zeroLoc)
    print(cellFaces)

    for i in range(orderStenc):
        polyStencSet[i], n_c, n_f = GTT.CentGhost(omega, order, cellFaces[i])
    
    polyStencSet = np.asarray(polyStencSet)

    IMat = np.eye(degFreed, degFreed)
    
    # YOU'RE GONNA NEED THESE TO RESTRICT FOR HIGHER EVEN ORDERS, TOO.
    
    
    polyMatU = IMat + 0
    
    
    mat = np.zeros((degFreed, degFreed), float)
    derivOp = mat + 0
    
    for d in range(orderStenc + 1):
        s = int(off - d)
        print('s:', s)
        
        derivMat = mat + 0
        np.fill_diagonal(derivMat, stenc[d])
        derivMat = np.roll(derivMat, s, axis = 0)
        
        polyMat = IMat + 0
        
        if (NU):
            if (s > 0):
                j = int(off - s)
                pAt = p
                pLow = (p - 1) % degFreed
                qAt = (q - s + 1) % degFreed #(q + 1) % degFreed
                for i in range (s):
                    polyMat[pAt, :] = 0
                    polyMat[pAt, pLow:pLow+2] = 0.5
                    polyMat[qAt, :] = polyStencSet[j, :]
                    pAt = (pAt - 1) % degFreed
                    pLow = (pLow - 2) % degFreed
                    qAt = (qAt + 1) % degFreed
                    j = int(j + 1)

            if (s < 0):
                j = int(off) # - s - 1
                qAt = (q + 1) % degFreed
                qLow = (q + 1) % degFreed
                pAt = (p + 1) % degFreed#p
                for i in range(abs(s)):
                    polyMat[qAt, :] = 0
                    polyMat[qAt, qLow:qLow+2] = 0.5
                    polyMat[pAt, :] = polyStencSet[j, :]
                    qAt = (qAt + 1) % degFreed
                    qLow = (qLow + 2) % degFreed
                    pAt = (pAt + 1) % degFreed
                    j = int(j + 1) # - 1
        
        matThis = derivMat @ polyMat
        print('Delta^{%d} =' %s)
        print(derivMat)
        print('A^{%d} =' %s)
        print(polyMat)
        print('D^{%d} =' %s)
        print(matThis)
        print('')
        
        derivOp = derivOp + matThis
    
    hMat = OT.StepMatrix(omega)
    
    
    derivOp = hMat @ derivOp
        
    return derivOp

In [9]:
if (diff == 'CD'):
    Ng = int(order / 2)
else:
    Ng = int((order + 1) / 2)
deriv = FaceOp(omegaAMR, order, diff, RL, Ng)
print('D =')
print(deriv)
print('')

[-0.5  0.5]
s: 0
Delta^{0} =
[[0.33333333 0.         0.         0.         0.         0.         0.         0.         0.         0.         0.         0.        ]
 [0.         0.33333333 0.         0.         0.         0.         0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.33333333 0.         0.         0.         0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.33333333 0.         0.         0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.33333333 0.         0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.33333333 0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.         0.33333333 0.         0.         0.         0.         0.        ]
 [0.         0.         0.

In [10]:
def SpaceDeriv1(omega, order, diff):
    errorLoc = 'ERROR:\nOperatorTools:\nMakeSpaceDeriv:\n'
    errorMess = ''
    if (diff == 'C' or diff == 'CD'):
        stenc = OT.CDStencil(order)
        if (order % 2 == 0):
            orderStenc = order
        else:
            orderStenc = int(order + 1)
        off = int(orderStenc / 2)
        loBound = -off / 2.
        hiBound = off / 2.
    else:
        orderStenc = order
        if (order % 2 == 0):
            orderStenc = int(order + 1)
        else:
            orderStenc = order
        off = ((orderStenc + 1) / 2)
        if (diff == 'U' or diff == 'UD'):
            stenc = OT.UDStencil(order)
            loBound = -off / 2.
            hiBound = (off - 1.) / 2.
        else:
            if (diff == 'D' or diff == 'DD'):
                stenc = OT.DDStencil(order)
                off = int(off - 1)
                loBound = -off / 2.
                hiBound = (off + 1.) / 2.
            else:
                errorMess = 'Invalid entry for variable diff. Must be \'C\', \'U\', \'D\' \'CD\', \'UD\', or \'DD\'.'
    if (errorMess != ''):
        sys.exit(errorLoc + errorMess)
    
#     stenc = np.ones(orderStenc + 1)
    
    degFreed = omega.degFreed
    hs = omega.h
    
    spots = np.roll(hs, -1) - hs
    
    if (all(spots == 0)):
        p = []
        q = []
        NU = False
    else:
        # Index before fine-coarse interface
        p = np.where(spots > 0)[0][0]
        # Index before coarse-fine interface
        q = np.where(spots < 0)[0][0]
        NU = True
    
    polyStencSet = [[] for i in range(orderStenc)]
    cellFaces = np.linspace(loBound, hiBound, num = orderStenc + 1)
    zeroLoc = np.where(cellFaces == 0)[0][0]
    cellFaces = np.delete(cellFaces, zeroLoc)

    for i in range(orderStenc):
        polyStencSet[i], n_c, n_f = GTT.CentGhost(omega, order, cellFaces[i])
    
    polyStencSet = np.asarray(polyStencSet)

    IMat = np.eye(degFreed, degFreed)
    
    # YOU'RE GONNA NEED THESE TO RESTRICT FOR HIGHER EVEN ORDERS, TOO.
    
    
    polyMatU = IMat + 0
    
    
    mat = np.zeros((degFreed, degFreed), float)
    derivOp = mat + 0
    
    for d in range(orderStenc + 1):
        s = int(off - d)
        print('s:', s)
        
        derivMat = mat + 0
        np.fill_diagonal(derivMat, stenc[d])
        derivMat = np.roll(derivMat, s, axis = 0)
        
        polyMat = IMat + 0
        
        if (NU):
            if (s > 0):
                j = int(off - s)
                pAt = p
                pLow = (p - 1) % degFreed
                qAt = (q - s + 1) % degFreed #(q + 1) % degFreed
                for i in range (s):
                    polyMat[pAt, :] = 0
                    polyMat[pAt, pLow:pLow+2] = 0.5
                    polyMat[qAt, :] = polyStencSet[j, :]
                    pAt = (pAt - 1) % degFreed
                    pLow = (pLow - 2) % degFreed
                    qAt = (qAt + 1) % degFreed
                    j = int(j + 1)

            if (s < 0):
                j = int(off) # - s - 1
                qAt = (q + 1) % degFreed
                qLow = (q + 1) % degFreed
                pAt = (p + 1) % degFreed#p
                for i in range(abs(s)):
                    polyMat[qAt, :] = 0
                    polyMat[qAt, qLow:qLow+2] = 0.5
                    polyMat[pAt, :] = polyStencSet[j, :]
                    qAt = (qAt + 1) % degFreed
                    qLow = (qLow + 2) % degFreed
                    pAt = (pAt + 1) % degFreed
                    j = int(j + 1) # - 1
        
        matThis = derivMat @ polyMat
        print('Delta^{%d} =' %s)
        print(derivMat)
        print('A^{%d} =' %s)
        print(polyMat)
        print('D^{%d} =' %s)
        print(matThis)
        print('')
        
        derivOp = derivOp + matThis
    
    hMat = OT.StepMatrix(omega)
    
    
    derivOp = hMat @ derivOp
        
    return derivOp

In [11]:
deriv = SpaceDeriv1(omegaAMR, order, diff)#, RL, Ng)
print('D =')
print(deriv)
print('')

s: 1
Delta^{1} =
[[ 0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.         -0.33333333]
 [-0.33333333  0.          0.          0.          0.          0.          0.          0.          0.          0.          0.          0.        ]
 [ 0.         -0.33333333  0.          0.          0.          0.          0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.         -0.33333333  0.          0.          0.          0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.         -0.33333333  0.          0.          0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.         -0.33333333  0.          0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.         -0.33333333  0.          0.          0.